### Configuration

Choose which scenario's to include in the comparison

In [1]:
scenarios = {
    'S1': {
        'load':         True,
        'reinitialize': True
    },
    'S2': {
        'load':         True,
        'reinitialize': True
    },
    'S3':{
        'load':         False,
        'reinitialize': False
    },
    'S4':{
        'load':         False,
        'reinitialize': False
    }
}

Choose which keys you want to save in the dataframe

In [264]:
regex = [
    # Energies
    'Eprim',
    'Eren',
    'Esto',
    
    'productionSite.Eboi',
    'productionSite.EhpEl',
    'productionSite.Esun',
    'productionSite.Ehp',
    'productionSite.ECHP',
    'productionSite.EelCHP',
    'productionSite.Esto',
    
    'grid.Egrid',
    'grid.Edis',
    'grid.Ehea',
    'grid.Esh',
    'grid.Edhw',
    'grid.EboosEl',
    'grid.Esto'
    
    # Powers
    'Pprim', 
    'Pren',
    'Qsto',
    
    'productionSite.Pboi',
    'productionSite.PhpEl',
    'productionSite.Qsun',
    'productionSite.Qhp',
    'productionSite.PCHP',
    'productionSite.PelCHP',
    'productionSite.Qsto',
    'productionSite.Qren',
    
    'grid.Qgrid',
    'grid.Qdis',
    'grid.Qhea',
    'grid.Qsh',
    'grid.Qdhw',
    'grid.Qhp',
    'grid.PboosEl',
    'grid.Qsto'
    
    ## Heatingsystems
    '.*TZones',
    '.*TZonem',
    '.*TZoned.*(?<!\))$',
    
    '.*TDhws',
    '.*TDhwm',
    '.*TDhwd.*(?<!\))$',
    
    '.*TSupplym',
    '.*TSupplys'
    
    '.*TGridi',
    '.*TGrido',

    '.*mRad',
    '.*mDhw',
    '.*mDhwReq'
    
    '.*tan.*vol.*T',
    ''
    
    # Grid
    '.*Pipe.*QLosses',
    '.*Pipe.*T.Out',
    
    
    # Production site
    '.*COP$',
    '.*modulation$',
    '.*mGrid$',
    'productionSite.solar.m_flow',
    'productionSite.boiler.m_flow',
    'productionSite.hp.m_flow',
    'productionSite.cHP.m_flow',
    
    '.*Tboi.* '
    
]

cumul = [
    'Eprim',
    'Eren',
    'Esto',
    'productionSite.Eboi',
    'productionSite.EhpEl',
    'productionSite.Esun',
    'productionSite.Ehp',
    'productionSite.EChpGas',
    'productionSite.EchpEl',
    'productionSite.Esto',
    'grid.Egrid',
    'grid.Edis',
    'grid.Ehea',
    'grid.Esh',
    'grid.Edhw',
    'grid.EboosEl',
    'grid.Esto'
]

Set the result file location

In [3]:
path = r'M:/De schipjes/Scenarios/'
name = r'year'
path = os.path.join(path, name)

### Imports

In [5]:
import os
import sys
import math

import pandas as pd

from datetime import date
from result import Result, Results, load

In [6]:
import charts
charts.default_settings.update(dict(show='tab', stock=True))

Server running in the folder c:\Users\u0098668\Documents\Modelica\DeSchipjes\notebooks\Analysis at 127.0.0.1:54650


### Initialization

In [7]:
periods = {
    '1': [[0, 31556926]],
    '2': [[0, 15778463], [15778463, 31556926]],
    '3': [[0, 10518976], [10518976, 21037951], [21037951, 31556926]]
}

In [218]:
def find_keys(regex, result):
    keys = []
    if type(regex) is not list:
        regex = [regex]
    
    for r in regex:
        for k in result.keys:
            if re.match(r, k):
                keys.append(k)
    return list(set(keys))

In [127]:
def load_result(scenario):
    # Find the files with their full path and load the results
    files = filter(lambda x: x.endswith('.mat'), 
        [os.path.join(path, scenario, f) for f in sorted(os.listdir(os.path.join(path, scenario)))])
    return [Result(f) for f in files]

def concat(results, regex):   
    # Get the number of files and calculate the start and stop times 
    n = len(results)
    period = math.ceil(31556926./3)
    
    durations = periods[str(n)]
    
    keys = find_keys(regex, results[0])
    
    # Load the dataframes
    dfs = [r.get(keys) for r in results]
    
    # Slice the correct times
    dfs = [dfs[i].truncate(
        before=date.fromtimestamp(durations[i][0]), 
        after=date.fromtimestamp(durations[i][1])) for i in range(0, n)]
    
    # Cumulate the results if necessary
    for i in range(1, n):
        for k in keys:
            if k in cumul:
                dfs[i][k] = dfs[i][k] + dfs[i-1][k][-1]
                
    # Concat the dataframes and save it
    return pd.concat(dfs) 

def load_df(scenario):
    return load(os.path.join(path, scenario, scenario + '.hdf'))

def save_df(df, scenario):
    dest = os.path.join(path, scenario, scenario + '.hdf')
    df.to_hdf(dest, 'data')
    
def 

In [ ]:
data = dict()
results = dict()
for scenario, config in scenarios.iteritems():
    if config['load']:
        if config['reinitialize']:
            results[scenario] = load_result(scenario)
            data[scenario] = concat(results[scenario], keys)
            save_df(data[scenario], scenario)
        else:
            data[scenario] = load_df(scenario)

Some further data manipulations...

In [276]:
if 'S1' in data:
    data['S1']['Eprim'] = data['S1']['Eprim'] + data['S1']['productionSite.ECHP']

In [281]:
loaded_scenarios = sorted(filter(lambda x: scenarios[x]['load'], scenarios.keys()))

In [312]:
months = []
for i in range(1,12):
    months.append([date(1970, i, 1), date(1970, i+1, 1)])

In [318]:
def monthly_diff(df, key):
    res = []
    
    for m in months:
        [i1, i2] = [df.index.searchsorted(m[0]), df.index.searchsorted(m[1])]
        [ix1, ix2] = [df.ix[df.index[i1]], df.ix[df.index[i2]]]
    
        res.append(ix2[key]-ix1[key])
        
    return res

### Plots

#### Total  Energy consumption / production

In [243]:
data['S1'] = df

In [287]:
def value_or_zero(data, s, y, i):
    try:
        return data[s][y][i]
    except KeyError:
        return 0

categories = [
    'Eprim',
    'Eren',
    'Esun',
    'productionSite.Eboi',
    'productionSite.EhpEl',
    'productionSite.Esun',
    'productionSite.Ehp',
    'productionSite.ECHP',
    'productionSite.EelCHP',
    'productionSite.Esto'
]

options = {
    'chart': {'zoomType': 'xy'},
    'title': {'text': 'Total energy consumption and production'},
    'xAxis': {
        'categories': categories
    },
    'yAxis': {'min': 0}
}

series = map(lambda s: {
    'name': s,
    'data': map(lambda y: value_or_zero(data, s, y, -1), categories)
}, loaded_scenarios)

charts.plot(
    type='column', 
    series=series, 
    options=options,
    stock=False,
    show='inline', 
    save=os.path.join(path, options['title']['text']) + '.html')

**Total heat usage**

In [283]:
categories = [
    'grid.Egrid',
    'grid.Edis',
    'grid.Ehea',
    'grid.Esh',
    'grid.Edhw',
    'grid.Esto',
    'grid.EboosEl'
]

options = {
    'chart': {'zoomType': 'xy'},
    'title': {'text': 'Heat usage in the grid'},
    'xAxis': {
        'categories': categories
    },
    'yAxis': {'min': 0}
}

series = map(lambda s: {
    'name': s,
    'data': map(lambda y: value_or_zero(data, s, y, -1), categories)
}, loaded_scenarios)

charts.plot(
    type='column', 
    series=series, 
    options=options,
    stock=False,
    show='inline', 
    save=os.path.join(path, options['title']['text']) + '.html')

In [286]:
losses = dict(zip(
    loaded_scenarios, 
    map(lambda x: data[x]['grid.Edis'][-1]/data[x]['grid.Egrid'][-1]*100, loaded_scenarios)))

categories = [
    'Year', 
    'januari', 
    'februari', 
    'maart', 
    'april', 
    'juni', 
    'juli',
    'augustus', 
    'september', 
    'oktober', 
    'november', 
    'december'
]
options = {
        'chart': {'zoomType': 'xy'},
        'title': {'text': categories[0]},
        'xAxis': {
            'categories': categories
        },
        'yAxis': {'min': 0, 'max': 10}
    }
series = map(lambda x: {
    'name': x,
    'data': [losses[x]]
}, loaded_scenarios)

charts.plot(
    type='column', 
    series=series, 
    options=options,
    stock=False,
    show='inline', 
    save=os.path.join(path, categories[0]) + '.html')

Possible explanations:
 - Losses are **not very dependent** of massflow. The higher the massflow, the more energy is distributed and the smaller the relative losses will be
 - S4 has the lowest energy needed due to the separate DHW production